In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
plates = pd.DataFrame({
    'data_path': [base_path/'2024.11.07_exp119'/'export', base_path/'2024.11.10_exp119_2'/'export'],
    'yaml_path': [base_path/'2024.11.07_exp119'/'export'/'wells.yaml']*2,
    'biorep': [1,2],
    'exp': ['exp119', 'exp119_2']
})
output_path = rd.rootdir/'output'/'new_piggybac'
cache_path = output_path/'new_piggybac.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A','YL1-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
data['marker'] = data['mGL-A']
data['output'] = data['YL1-A']

In [ ]:
plot_df = data.groupby(['biorep','construct']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='output', hue='construct', palette=main_palette, kind='kde',
                row='biorep', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=8)

for (biorep, construct), ax in g.axes_dict.items():
    ax.axvline(2e2, c='black', ls=':', zorder=0)

In [ ]:
gates = pd.DataFrame({
    'mRuby2-A': [2e2]*2,
    'YL1-A': [2e2]*2,
    'mGL-A': [2e2]*2,
    'biorep': [1,2],
    'exp': ['exp119', 'exp119_2']
})

gates['marker'] = 'mGL-A'
gates['output'] = 'YL1-A'
display(gates)

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing'])]
display(df)

In [ ]:
plot_df = df.groupby(['biorep','construct']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='output', hue='construct', palette=main_palette, kind='kde',
                row='biorep', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=8)

for (biorep, construct), ax in g.axes_dict.items():
    ax.axvline(2e2, c='black', ls=':', zorder=0)

In [ ]:
plot_df = df.groupby(['biorep','construct']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='mRuby2-A', hue='construct', palette=main_palette, kind='kde',
                row='biorep', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=8)

for (biorep, construct), ax in g.axes_dict.items():
    ax.axvline(2e2, c='black', ls=':', zorder=0)